# Initial Exploration of the Zhvi_AllHomes and TRI 2011-2015 US Data

An initial peek at the data present in the Zillow and TRI sets, doing a simple comparison of average home prices in a zip code and the total number of TRI reports for that zip code.

In [1]:
%pylab inline
import numpy, pandas
# The below option suppresses a warning about making changes diretly to a DataFrame
pandas.options.mode.chained_assignment = None

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read in the relevant data files
zhvi = pandas.read_csv("DATA/ZILLOW_DATA/Zip_Zhvi_AllHomes.csv")

tri2011 = pandas.read_csv("DATA/TRI_DATA/TRI_2011_US.csv", dtype={'NAICS_2': object, 'NAICS_3': object, 'NAICS_4': object, 'NAICS_5': object, 'PARENT_COMPANY_DB_NUMBER': object})
tri2012 = pandas.read_csv("DATA/TRI_DATA/TRI_2012_US.csv", dtype={'NAICS_2': object, 'NAICS_3': object, 'NAICS_4': object, 'NAICS_5': object, 'PARENT_COMPANY_DB_NUMBER': object})
tri2013 = pandas.read_csv("DATA/TRI_DATA/TRI_2013_US.csv", dtype={'TRIBE': object, 'PARENT_COMPANY_DB_NUMBER': object})
tri2014 = pandas.read_csv("DATA/TRI_DATA/TRI_2014_US.csv", dtype={'PARENT_COMPANY_DB_NUMBER': object})
tri2015 = pandas.read_csv("DATA/TRI_DATA/TRI_2015_US.csv", dtype={'PARENT_COMPANY_DB_NUMBER': object})

triReports = [tri2011, tri2012, tri2013, tri2014, tri2015]

## Cleaning up the TRI data

A few notes on the ZIP codes found in the TRI datasets:

If a zip code has leading 0's in it, pandas drops the 0's as it reads in the data.
For example, 04276, the ZIP for Rumford, ME becomes 4276. Any ZIP shorter than 5 digits should be read as having leading 0's to bring it to 5 digits

In some cases, full 9 digit ZIPs are recorded. The trailing 4 digits are used to provide further specificity to the Postal Service within the leading 5 digit ZIP, and as such are not needed for our purposes. The method below takes a series of ZIPs from a TRI dataset and reformats any 9 digit ZIPs to 5 digits

### Sorting and Organizing
Once the ZIPs have been cleaned up, I'm re-ordering the data by ZIP code, as well as counting the total number of reports filed in a zip code per year in the TRI data

In [10]:
triNC = []

for year in triReports:
    triNC.append(year[(year['ST'] == 'NC')].copy())

In [4]:
def cleanTriZIP(zips):
    """
    Re-formats 9 digit zip codes in the TRI dataset to 5 digit zip codes
    """
    for i in range(len(zips)):
        if zips[i] > 99999:
            zips[i] = numpy.int32((zips[i]/10000))

In [15]:
#Reformat 9 digit zips to 5 digits

triZIPs = []

for year in triNC:
    triZIPs.append(year.ZIP)

# triZIPs.append(tri2011.ZIP)
# triZIPs.append(tri2012.ZIP)
# triZIPs.append(tri2013.ZIP)
# triZIPs.append(tri2014.ZIP)
# triZIPs.append(tri2015.ZIP)

for z in triZIPs:
    cleanTriZIP(z.values)

In [16]:
sortedTriZIPs = []

for s in triZIPs:
    sortedTriZIPs.append(s.groupby(s.values).size())
    sortedTriZIPs[-1].index.name = 'ZIP'
    sortedTriZIPs[-1].name = 'REPORTS'

In [47]:
sortedTriZIPs

[ZIP
 27013    12
 27017     1
 27021     2
 27025     7
 27028     6
 27030    10
 27042    12
 27045     6
 27050     2
 27052    18
 27101     2
 27103     5
 27105    10
 27107    21
 27203    15
 27205     8
 27209     3
 27212     3
 27214     2
 27215     5
 27217    10
 27229     9
 27233     1
 27235     1
 27253     4
 27260    45
 27262    29
 27263    16
 27278     1
 27282    11
          ..
 28690     1
 28694     4
 28697     3
 28704    28
 28711     2
 28713     1
 28716    23
 28726     8
 28728     2
 28730     1
 28732    13
 28734     3
 28748     1
 28752     5
 28754     1
 28758     4
 28759     1
 28762     2
 28771     4
 28777     2
 28779     3
 28787     7
 28791     2
 28792    22
 28803    10
 28804    11
 28806     2
 28901     1
 28904     1
 28906     2
 Name: REPORTS, Length: 302, dtype: int64, ZIP
 27013    12
 27017     2
 27021     2
 27025     7
 27028     6
 27030    10
 27042    12
 27045     6
 27050     2
 27052    21
 27101     3
 27103     5

## Cleaning up the Zhvi_AllHomes data

My initial feeling is that the Zhvi dataset is much easier to work with. However, one change is that I have re-indexed the DataFrame so that each rows index is the ZIP code that it represents

In [18]:
# Re-index the zhvi DataFrame based off of zip code, which is called RegionName in the data set
zhvi.index = zhvi['RegionName']

### Dividing the Data
For this initial dive into the data, I am pulling out specifically the Zhvi data from NC, and looking between the years 2011-2015. For each year, I am taking the mean value from the Zhvi, since the data from the TRI is tracked annually

In [19]:
# From the initial DataFrame, copy out the data from NC
zhviNCar = zhvi[(zhvi['State'] == 'NC')].copy()

#Sort the zipcodes in ascending order
zhviNCar.sort_values('RegionName', inplace=True)

#Drop all columns outside of the 2011-2015 time frame
zhviNCar = zhviNCar.loc[:, '2011-01':'2015-12']

#Find the yearly mean prices for the zip codes
zhviNCarMeans = []
zhviNCarMeans.append(zhviNCar.T.iloc[0:12].mean())
zhviNCarMeans.append(zhviNCar.T.iloc[12:24].mean())
zhviNCarMeans.append(zhviNCar.T.iloc[24:36].mean())
zhviNCarMeans.append(zhviNCar.T.iloc[36:48].mean())
zhviNCarMeans.append(zhviNCar.T.iloc[48:60].mean())

## Finding the Overlap
There are ZIPs in the ZHVI data that aren't in the TRI data, and vice versa. To help draw meaningful conclusions it might be useful to find where the two data sets overlap

In [55]:
masterFrame = pandas.DataFrame()
masterFrame['2011mean'] = zhviNCarMeans[0].values
masterFrame['2012mean'] = zhviNCarMeans[1].values
masterFrame['2013mean'] = zhviNCarMeans[2].values
masterFrame['2014mean'] = zhviNCarMeans[3].values
masterFrame['2015mean'] = zhviNCarMeans[4].values
masterFrame.index = zhviNCarMeans[0].index
masterFrame.index.rename('ZIP', True)

for triYears in sortedTriZIPs:
    for entry in triYears.index:
        if entry in masterFrame.index:
            print 'y'
        else:
            print 'thats a big fat no'


y
thats a big fat no
y
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
y
y
y
y
y
y
y
y
y
thats a big fat no
thats a big fat no
y
y
y
thats a big fat no
y
y
y
y
y
y
y
y
y
thats a big fat no
y
y
y
y
y
thats a big fat no
y
y
thats a big fat no
y
y
thats a big fat no
y
y
thats a big fat no
y
thats a big fat no
thats a big fat no
y
thats a big fat no
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
thats a big fat no
thats a big fat no
y
y
y
y
y
y
y
thats a big fat no
y
thats a big fat no
thats a big fat no
y
y
y
thats a big fat no
y
y
y
y
y
y
y
thats a big fat no
y
y
y
thats a big fat no
y
thats a big fat no
y
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
y
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
y
thats a big fat no
thats a big fat no
thats a big fat no
y
y
thats a big fat no
y
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
thats a big fat no
that

In [45]:
masterFrame

,2011mean,2012mean,2013mean,2014mean,2015mean
ZIP,,,,,
27009,139900.000000,137358.333333,139641.666667,139825.000000,143183.333333
27012,181325.000000,179141.666667,180758.333333,185883.333333,189391.666667
27013,123258.333333,122633.333333,123750.000000,123991.666667,126800.000000
27016,110916.666667,109958.333333,109575.000000,111358.333333,107941.666667
27019,129516.666667,128950.000000,129566.666667,129208.333333,131258.333333
27021,124691.666667,125341.666667,122575.000000,122758.333333,123958.333333
27022,114116.666667,112825.000000,115033.333333,113291.666667,113666.666667
27023,176941.666667,173966.666667,175891.666667,179575.000000,181841.666667
27040,153066.666667,149608.333333,149500.000000,150825.000000,152008.333333


In [ ]:
def buildMasterZIP(tri, zhvi):
#     result = pandas.DataFrame({'2011Mean': NaN, '2012Mean': NaN, '2013Mean': NaN, '2014Mean': NaN, '2015Mean': NaN}, zhvi[0].index)
    masterFrame = pandas.DataFrame()


    return result

In [ ]:
test = buildMasterZIP(sortedTriZIPs, zhviNCarMeans)
test

In [ ]:
def findOverlap(tri, zhvi):
    """
    Takes equally sized lists as arguments. Each list is a series that is indexed by ZIP codes.
    Method returns two lists, indicating where one overlaps the other. For now, kept in two lists
    to make graphing easier. I would like to put this all in one dataframe but I'll figure that out later
    """
    #first list returns TRI data values that overlap the ZHVI data values
    TRIinZHVI = []
    #second list does the opposite.
    ZHVIinTRI = []
    
    for i in range(len(tri)):
        mask = zhvi[i].index.isin(tri[0].index)
        ZHVIinTRI.append(zhvi[i].where(mask).dropna())
        
        mask = tri[i].index.isin(ZHVIinTRI[i].index)
        TRIinZHVI.append(tri[i].where(mask).dropna())
    
    return TRIinZHVI, ZHVIinTRI

In [ ]:
triNC, zhviTriNC = findOverlap(sortedTriZIPs, zhviNCarMeans)

### Graphing the Data

The first graph below shows the mean annual price per ZIP code from the Zhvi_AllHomes dataset
The second graph is the total number of TRI reports filed that same year in the same ZIP code.

I only am showing the first 20 here to give an idea of what might be possible. There are far to many records to actually show in one graph so we'll need to find a way to usefully show this data

In [ ]:
fig, ax = subplots(figsize = (18,5))

index = arange(20)
width = .15
offset = 0

for s in zhviTriNC:
    ax.bar(index + (width*offset), s.values[7:27], width)
    offset += 1

ax.set_xticks(index+(width*2))
ax.set_xticklabels(zhviTriNC[0].index[7:27])
ax.set_title('Annual Mean Home Value in selected NC zipcodes 2011-2015')
ax.set_xlabel('Zip Code')
ax.set_ylabel('Value')
ax.legend(['2011', '2012', '2013', '2014', '2015']);

In [ ]:
# fig, ax = subplots(figsize = (18,5))

# index = arange(20)
# width = .15
# offset = 0

# for r in triNC:
#     ax.bar(index + (width*offset), r.values[7:27], width)
#     offset += 1

# ax.set_xticks(index+(width*2))
# ax.set_xticklabels(triNC[0].index[7:27])
# ax.set_title('Total TRI reports per ZIP code 2011-2015')
# ax.set_xlabel('Zip Code')
# ax.set_ylabel('Report Count')
# ax.legend(['2011', '2012', '2013', '2014', '2015']);

## More to come...

Well since my first graphs were totally wrong, I dont know what to say. The Mean Home Value has been corrected but the TRI report numbers are still off, just less off than they were before. This has been an important lesson for me in double checking your work before shooting your mouth off about cool results.